In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("Tags and Title for every movie for 2015")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Tags_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/tag.csv")
      )
Movies_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/movie.csv")
      )
Tags_Dataframe=Tags_Dataframe.withColumn("timestamp",date_format("timestamp","yyy"))

Tagdf=(Tags_Dataframe.where("timestamp==2015")
                     .drop('userId')
                     .join(Movies_Dataframe,Movies_Dataframe.movieId==Tags_Dataframe.movieId)
                     .drop(Movies_Dataframe.movieId)
                     .drop(Movies_Dataframe.genres)   
      )

Tagdf=Tagdf.groupBy("title").agg(collect_list("tag"))

Tagdf.withColumn("tags", concat_ws(",", "collect_list(tag)")) \
     .drop("collect_list(tag)")\
     .sort('title',ascending=True)\
     .show(n=5,truncate=0)
stagemetrics.end()

+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                             |tags                                                                                                                                                                                                                                                                                                                                                                  |
+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 7
numTasks => 212
elapsedTime => 25422 (25 s)
stageDuration => 23462 (23 s)
executorRunTime => 54191 (54 s)
executorCpuTime => 14342 (14 s)
executorDeserializeTime => 13305 (13 s)
executorDeserializeCpuTime => 3014 (3 s)
resultSerializationTime => 261 (0.3 s)
jvmGCTime => 1163 (1 s)
shuffleFetchWaitTime => 37 (37 ms)
shuffleWriteTime => 398 (0.4 s)
resultSize => 1451231 (1417.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 11116832
recordsRead => 985685
bytesRead => 47011272 (44.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 4972
shuffleTotalBlocksFetched => 778
shuffleLocalBlocksFetched => 778
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 632645 (617.0 KB)
shuffleLocalBytesRead => 632645 (617.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuff